In [1]:
import cvxpy
import random
import numpy as np
import pandas as pd

In [2]:
# サンプルデータの読み込み
sample = pd.read_csv('sample_3*30.csv')

In [3]:
sample.head(3)

,id,price_A,price_B,price_C,rating_A,rating_B,rating_C
0,1,380,385,389,4.1,3.3,4.0
1,2,943,947,961,10.1,8.9,9.3
2,3,980,980,987,10.4,8.7,10.3


In [4]:
# クライアトa, b, cの元々持っていた枠をランダムに決める。
length = len(sample)
ids = list(range(length))
# idは1からなので、1を足す。
random_ids = [id+1 for id in ids]

# Break Point 1の略。
bp1 = length//3 
bp2 = length//3 + bp1

random.shuffle(random_ids)
id_A, id_B, id_C = random_ids[:bp1], random_ids[bp1:bp2], random_ids[bp2:]

In [5]:
# それぞれのidを表示。 ランダムにシャッフルされていることがわかる。
print(id_A)
print(id_B)
print(id_C)

[13, 19, 8, 11, 17, 20, 26, 1, 15, 28]
[2, 10, 25, 24, 18, 14, 7, 16, 12, 21]
[23, 3, 30, 4, 29, 22, 5, 27, 9, 6]


In [6]:
# それぞれのidのデータのみを取り出すための判別用カラム。
sample['judge_A'] = sample['id'].apply(lambda x:x in id_A)
sample['judge_B'] = sample['id'].apply(lambda x:x in id_B)
sample['judge_C'] = sample['id'].apply(lambda x:x in id_C)

In [7]:
# それぞれのidのデータを取得。
df_A = sample[sample['judge_A']].loc[:, ['id','price_A', 'rating_A']]
df_B = sample[sample['judge_B']].loc[:, ['id','price_B', 'rating_B']]
df_C = sample[sample['judge_C']].loc[:, ['id','price_C', 'rating_C']]

In [8]:
# 判別用カラムを削除。
del sample['judge_A']
del sample['judge_B']
del sample['judge_C']

In [9]:
# 元々の値段の総和、元々の視聴率の和を計算する。
# Original price, Original point
original_price_A, original_point_A = sum(df_A['price_A']), sum(df_A['rating_A'])
original_price_B, original_point_B = sum(df_B['price_B']), sum(df_B['rating_B'])
original_price_C, original_point_C = sum(df_C['price_C']), sum(df_C['rating_C'])

In [10]:
# 獲得した枠のidを貯めておく。
obtained_id_A = []
obtained_id_B = []
obtained_id_C = []

In [11]:
# 獲得した枠の価格を貯めていく。
obtained_price_A = 0
obtained_price_B = 0
obtained_price_C = 0

In [12]:
# 獲得した枠の視聴率を貯めていく。
obtained_point_A = 0
obtained_point_B = 0
obtained_point_C = 0

In [13]:
# ここに結果を格納していき、何度も繰り返す。（視聴率和の合計値が'key'で、その時のidが'value'）
result = dict()

In [14]:
# 各クライアントの考える価値を数値化し、表に加える。
# 指定した視聴率/そのクライアントの号数価格
sample['value_A'] = round((sample['rating_A']/sample['price_A'])*1000, 2)
sample['value_B'] = round((sample['rating_B']/sample['price_B'])*1000, 2)
sample['value_C'] = round((sample['rating_C']/sample['price_C'])*1000, 2)

In [15]:
print('client_Aの交換前の合計価格:', original_price_A, 'client_Aの交換前の合計視聴率', original_point_A)
print('client_Bの交換前の合計価格:', original_price_B, 'client_Bの交換前の合計視聴率', original_point_B)
print('client_Cの交換前の合計価格:', original_price_C, 'client_Cの交換前の合計視聴率', original_point_C)
sample.head(3)

client_Aの交換前の合計価格: 6751 client_Aの交換前の合計視聴率 65.3
client_Bの交換前の合計価格: 6826 client_Bの交換前の合計視聴率 68.1
client_Cの交換前の合計価格: 5409 client_Cの交換前の合計視聴率 53.8


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,2,943,947,961,10.1,8.9,9.3,10.71,9.40,9.68
2,3,980,980,987,10.4,8.7,10.3,10.61,8.88,10.44


<b><font color='Red'>※valueは、基本的に正の数じゃないと、選ばなくなってしまう（０の方が大きいから。）</font></b>

***
***

### 〜ここからアルゴリズム〜

In [16]:
# ナップザック問題。

# size = 値段のデータ。
# weight = 価値のデータ。
# capacity = 元々の値段×1.05倍の値段。
# ids = 取得すべきだと考えられた枠のid    

def Knapsack(size, weight, capacity, obtained=0, rate=0.05):
    # 価格が元の価格の1.05%を超えても、終わり。
    if capacity*(1 + rate) - obtained > 0:        
        x = cvxpy.Variable(size.shape[0], boolean=True)
        count = sum(np.ones(len(size)) * x) # 要素の個数(これを加えて平均を最大化しないと、価格の小さいものばかりをとってしまう。)
        objective = cvxpy.Maximize(weight * x / count) # 目的
        constraints = [capacity*(1 + rate) -  obtained >= size * x]# 制限。
        constraints += [capacity*(1 - rate) - obtained <= size * x]# ここでは、capacity(価格)の上下5%を取っている。

        prob = cvxpy.Problem(objective, constraints)
        prob.solve(solver=cvxpy.ECOS_BB)
        result = [round(ix, 0) for ix in x.value]

        return result
    else:
        return []

***
***
<b>〜ループ開始〜</b>
***
***

<b><font size=5><font color='#000000'>ループ①</font></font></b>

In [17]:
# エラー処理① Knapsack問題で、capacityの枠内に収まる解が見つからない場合。
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = original_price_A

rate=0.05
if obtained_point_A < original_point_A:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_a = [0 for i in range(len(size_A))]
else:
    ids_a = [0 for i in range(len(size_A))]

In [18]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = original_price_B

rate=0.05
if obtained_point_B < original_point_B: # 獲得した枠の合計視聴率が元々の合計視聴率よりも低い場合。
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_b = [0 for i in range(len(size_B))]
else: 
        ids_b = [0 for i in range(len(size_B))]

In [19]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = original_price_C

rate=0.05
if obtained_point_C < original_point_C:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]
else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]

In [20]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [21]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

6.0 個の枠: [-1.  1.  2. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 13. -1. -1. 16. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. 27. 28. -1.]
6.0 個の枠: [-1.  1.  2. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 13. -1. -1. 16. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. 27. 28. -1.]
5.0 個の枠: [-1. -1.  2. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 13. -1. -1. 16. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. 27. 28. -1.]


In [22]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [23]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [24]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: [1, 16, 28]
client_b: [13]
client_c: [2, 27]


In [25]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [26]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [27]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [28]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [29]:
print('残りの枠:', len(sample))
print('*'*45)
print('【client_A(価格)】', '交換前:', round(original_price_A,0), '獲得:', round(obtained_price_A,0)) 
print('【client_B(価格)】', '交換前:', round(original_price_B,0), '獲得:', round(obtained_price_B,0))
print('【client_C(価格)】', '交換前:', round(original_price_C,0), '獲得:', round(obtained_price_C,0))
print('*'*45)
print('【client_A(視聴率)】', '交換前:', round(original_point_A,1), '獲得:', round(obtained_point_A,1), '○' if obtained_point_A > original_point_A else '×')
print('【client_B(視聴率)】', '交換前:', round(original_point_B,1), '獲得:', round(obtained_point_B,1), '○' if obtained_point_B > original_point_B else '×')
print('【client_C(視聴率)】', '交換前:', round(original_point_C,1), '獲得:', round(obtained_point_C,1), '○' if obtained_point_C > original_point_C else '×')
display(sample.head(3))

残りの枠: 24
*********************************************
【client_A(価格)】 交換前: 6751 獲得: 3273
【client_B(価格)】 交換前: 6826 獲得: 1058
【client_C(価格)】 交換前: 5409 獲得: 2273
*********************************************
【client_A(視聴率)】 交換前: 65.3 獲得: 33.7 ×
【client_B(視聴率)】 交換前: 68.1 獲得: 11.3 ×
【client_C(視聴率)】 交換前: 53.8 獲得: 23.8 ×


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
2,5,167,149,178,1.5,1.5,2.0,8.98,10.07,11.24


In [30]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

***
***

<b><font size=5><font color='#222222'>ループ②</font></font></b>

In [31]:
# エラー処理① Knapsack問題で、capacityの枠内に収まる解が見つからない場合。
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = original_price_A

rate=0.05
if obtained_point_A < original_point_A:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_a = [0 for i in range(len(size_A))]
else:
    ids_a = [0 for i in range(len(size_A))]

In [32]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = original_price_B

rate=0.05
if obtained_point_B < original_point_B: # 獲得した枠の合計視聴率が元々の合計視聴率よりも低い場合。
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_b = [0 for i in range(len(size_B))]
else: 
        ids_b = [0 for i in range(len(size_B))]

In [33]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = original_price_C

rate=0.05
if obtained_point_C < original_point_C:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]
else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]

In [34]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [35]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

4.0 個の枠: [-1. -1. -1. -1.  4. -1.  6. -1. -1.  9. -1. -1. -1. -1. -1. -1. 16. -1.
 -1. -1. -1. -1. -1. -1.]
8.0 個の枠: [-1. -1. -1. -1.  4.  5.  6. -1.  8.  9. -1. -1. -1. -1. 14. -1. 16. -1.
 -1. -1. -1. 21. -1. -1.]
4.0 個の枠: [-1. -1. -1. -1.  4. -1.  6. -1. -1.  9. -1. -1. -1. -1. -1. -1. 16. -1.
 -1. -1. -1. -1. -1. -1.]


In [36]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [37]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [38]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: [4, 6, 9, 16]
client_b: [5, 8, 14, 21]
client_c: []


In [39]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [40]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [41]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [42]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [43]:
print('残りの枠:', len(sample))
print('*'*45)
print('【client_A(価格)】', '交換前:', round(original_price_A,0), '獲得:', round(obtained_price_A,0)) 
print('【client_B(価格)】', '交換前:', round(original_price_B,0), '獲得:', round(obtained_price_B,0))
print('【client_C(価格)】', '交換前:', round(original_price_C,0), '獲得:', round(obtained_price_C,0))
print('*'*45)
print('【client_A(視聴率)】', '交換前:', round(original_point_A,1), '獲得:', round(obtained_point_A,1), '○' if obtained_point_A > original_point_A else '×')
print('【client_B(視聴率)】', '交換前:', round(original_point_B,1), '獲得:', round(obtained_point_B,1), '○' if obtained_point_B > original_point_B else '×')
print('【client_C(視聴率)】', '交換前:', round(original_point_C,1), '獲得:', round(obtained_point_C,1), '○' if obtained_point_C > original_point_C else '×')
display(sample.head(3))

残りの枠: 16
*********************************************
【client_A(価格)】 交換前: 6751 獲得: 6620
【client_B(価格)】 交換前: 6826 獲得: 3779
【client_C(価格)】 交換前: 5409 獲得: 2273
*********************************************
【client_A(視聴率)】 交換前: 65.3 獲得: 67.0 ○
【client_B(視聴率)】 交換前: 68.1 獲得: 39.8 ×
【client_C(視聴率)】 交換前: 53.8 獲得: 23.8 ×


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
2,5,167,149,178,1.5,1.5,2.0,8.98,10.07,11.24


In [44]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

***
***

<b><font size=5><font color='#444444'>ループ③</font></font></b>

In [45]:
# エラー処理① Knapsack問題で、capacityの枠内に収まる解が見つからない場合。
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = original_price_A

rate=0.05
if obtained_point_A < original_point_A:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_a = [0 for i in range(len(size_A))]
else:
    ids_a = [0 for i in range(len(size_A))]

In [46]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = original_price_B

rate=0.05
if obtained_point_B < original_point_B: # 獲得した枠の合計視聴率が元々の合計視聴率よりも低い場合。
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_b = [0 for i in range(len(size_B))]
else: 
        ids_b = [0 for i in range(len(size_B))]

In [47]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = original_price_C

rate=0.05
if obtained_point_C < original_point_C:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]
else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]

In [48]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [49]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

0 個の枠: [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
5.0 個の枠: [-1. -1. -1. -1. -1.  5. -1. -1. -1. -1. -1. -1. 12. 13. 14. 15.]
5.0 個の枠: [-1. -1. -1. -1. -1.  5. -1. -1. -1. -1. -1. -1. 12. 13. 14. 15.]


In [50]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [51]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [52]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: []
client_b: [12, 14]
client_c: [5, 13, 15]


In [53]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [54]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [55]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [56]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [57]:
print('残りの枠:', len(sample))
print('*'*45)
print('【client_A(価格)】', '交換前:', round(original_price_A,0), '獲得:', round(obtained_price_A,0)) 
print('【client_B(価格)】', '交換前:', round(original_price_B,0), '獲得:', round(obtained_price_B,0))
print('【client_C(価格)】', '交換前:', round(original_price_C,0), '獲得:', round(obtained_price_C,0))
print('*'*45)
print('【client_A(視聴率)】', '交換前:', round(original_point_A,1), '獲得:', round(obtained_point_A,1), '○' if obtained_point_A > original_point_A else '×')
print('【client_B(視聴率)】', '交換前:', round(original_point_B,1), '獲得:', round(obtained_point_B,1), '○' if obtained_point_B > original_point_B else '×')
print('【client_C(視聴率)】', '交換前:', round(original_point_C,1), '獲得:', round(obtained_point_C,1), '○' if obtained_point_C > original_point_C else '×')
display(sample.head(3))

残りの枠: 11
*********************************************
【client_A(価格)】 交換前: 6751 獲得: 6620
【client_B(価格)】 交換前: 6826 獲得: 4993
【client_C(価格)】 交換前: 5409 獲得: 4079
*********************************************
【client_A(視聴率)】 交換前: 65.3 獲得: 67.0 ○
【client_B(視聴率)】 交換前: 68.1 獲得: 52.5 ×
【client_C(視聴率)】 交換前: 53.8 獲得: 40.5 ×


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,1,380,385,389,4.1,3.3,4.0,10.79,8.57,10.28
1,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
2,5,167,149,178,1.5,1.5,2.0,8.98,10.07,11.24


In [58]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

***
***

<b><font size=5><font color='#666666'>ループ④</font></font></b>

In [59]:
# エラー処理① Knapsack問題で、capacityの枠内に収まる解が見つからない場合。
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = original_price_A

rate=0.05
if obtained_point_A < original_point_A:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_a = [0 for i in range(len(size_A))]
else:
    ids_a = [0 for i in range(len(size_A))]

In [60]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = original_price_B

rate=0.05
if obtained_point_B < original_point_B: # 獲得した枠の合計視聴率が元々の合計視聴率よりも低い場合。
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_b = [0 for i in range(len(size_B))]
else: 
        ids_b = [0 for i in range(len(size_B))]

In [61]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = original_price_C

rate=0.05
if obtained_point_C < original_point_C:
    for try_count in range(6): #5回までなら、rateを大きくして再度試せる。
        try:
            ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]
else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]

In [62]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [63]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

0 個の枠: [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
5.0 個の枠: [ 0. -1. -1. -1.  4. -1. -1.  7.  8. -1. 10.]
4.0 個の枠: [ 0. -1. -1. -1.  4. -1. -1.  7.  8. -1. -1.]


In [64]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [65]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [66]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: []
client_b: [0, 4, 7, 8, 10]
client_c: []


In [67]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [68]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [69]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [70]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [71]:
print('残りの枠:', len(sample))
print('*'*45)
print('【client_A(価格)】', '交換前:', round(original_price_A,0), '獲得:', round(obtained_price_A,0)) 
print('【client_B(価格)】', '交換前:', round(original_price_B,0), '獲得:', round(obtained_price_B,0))
print('【client_C(価格)】', '交換前:', round(original_price_C,0), '獲得:', round(obtained_price_C,0))
print('*'*45)
print('【client_A(視聴率)】', '交換前:', round(original_point_A,1), '獲得:', round(obtained_point_A,1), '○' if obtained_point_A > original_point_A else '×')
print('【client_B(視聴率)】', '交換前:', round(original_point_B,1), '獲得:', round(obtained_point_B,1), '○' if obtained_point_B > original_point_B else '×')
print('【client_C(視聴率)】', '交換前:', round(original_point_C,1), '獲得:', round(obtained_point_C,1), '○' if obtained_point_C > original_point_C else '×')
display(sample.head(3))

残りの枠: 6
*********************************************
【client_A(価格)】 交換前: 6751 獲得: 6620
【client_B(価格)】 交換前: 6826 獲得: 6926
【client_C(価格)】 交換前: 5409 獲得: 4079
*********************************************
【client_A(視聴率)】 交換前: 65.3 獲得: 67.0 ○
【client_B(視聴率)】 交換前: 68.1 獲得: 72.4 ○
【client_C(視聴率)】 交換前: 53.8 獲得: 40.5 ×


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,4,217,237,227,2.3,1.9,2.3,10.60,8.02,10.13
1,5,167,149,178,1.5,1.5,2.0,8.98,10.07,11.24
2,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55


In [72]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

***
***

<b><font size=5><font color='#888888'>ループ⑤</font></font></b>

In [73]:
# エラー処理① Knapsack問題で、capacityの枠内に収まる解が見つからない場合。
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = original_price_A

rate=0.05
if obtained_point_A < original_point_A:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_a = [0 for i in range(len(size_A))]
else:
    ids_a = [0 for i in range(len(size_A))]

In [74]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = original_price_B

rate=0.05
if obtained_point_B < original_point_B: # 獲得した枠の合計視聴率が元々の合計視聴率よりも低い場合。
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_b = [0 for i in range(len(size_B))]
else: 
        ids_b = [0 for i in range(len(size_B))]

In [75]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = original_price_C

rate=0.05
if obtained_point_C < original_point_C:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]
else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]

In [76]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [77]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

0 個の枠: [-1 -1 -1 -1 -1 -1]
0 個の枠: [-1 -1 -1 -1 -1 -1]
5.0 個の枠: [ 0.  1. -1.  3.  4.  5.]


In [78]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [79]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [80]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: []
client_b: []
client_c: [0, 1, 3, 4, 5]


In [81]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [82]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [83]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [84]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [85]:
print('残りの枠:', len(sample))
print('*'*45)
print('【client_A(価格)】', '交換前:', round(original_price_A,0), '獲得:', round(obtained_price_A,0)) 
print('【client_B(価格)】', '交換前:', round(original_price_B,0), '獲得:', round(obtained_price_B,0))
print('【client_C(価格)】', '交換前:', round(original_price_C,0), '獲得:', round(obtained_price_C,0))
print('*'*45)
print('【client_A(視聴率)】', '交換前:', round(original_point_A,1), '獲得:', round(obtained_point_A,1), '○' if obtained_point_A > original_point_A else '×')
print('【client_B(視聴率)】', '交換前:', round(original_point_B,1), '獲得:', round(obtained_point_B,1), '○' if obtained_point_B > original_point_B else '×')
print('【client_C(視聴率)】', '交換前:', round(original_point_C,1), '獲得:', round(obtained_point_C,1), '○' if obtained_point_C > original_point_C else '×')
display(sample.head(3))

残りの枠: 1
*********************************************
【client_A(価格)】 交換前: 6751 獲得: 6620
【client_B(価格)】 交換前: 6826 獲得: 6926
【client_C(価格)】 交換前: 5409 獲得: 5308
*********************************************
【client_A(視聴率)】 交換前: 65.3 獲得: 67.0 ○
【client_B(視聴率)】 交換前: 68.1 獲得: 72.4 ○
【client_C(視聴率)】 交換前: 53.8 獲得: 53.6 ×


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,6,137,155,152,1.4,1.4,1.3,10.22,9.03,8.55


In [86]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

***
***

<b><font size=5><font color='#888888'>ループ⑥</font></font></b>

In [87]:
# エラー処理① Knapsack問題で、capacityの枠内に収まる解が見つからない場合。
size_A = np.array(sample['price_A'])
weight_A = np.array(sample['value_A'])
capacity_A = original_price_A

rate=0.05
if obtained_point_A < original_point_A:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_a = Knapsack(size_A, weight_A, capacity_A,  obtained_price_A, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_a = [0 for i in range(len(size_A))]
else:
    ids_a = [0 for i in range(len(size_A))]

In [88]:
size_B = np.array(sample['price_B'])
weight_B = np.array(sample['value_B'])
capacity_B = original_price_B

rate=0.05
if obtained_point_B < original_point_B: # 獲得した枠の合計視聴率が元々の合計視聴率よりも低い場合。
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_b = Knapsack(size_B, weight_B, capacity_B,  obtained_price_B, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_b = [0 for i in range(len(size_B))]
else: 
        ids_b = [0 for i in range(len(size_B))]

In [89]:
size_C = np.array(sample['price_C'])
weight_C = np.array(sample['value_C'])
capacity_C = original_price_C

rate=0.05
if obtained_point_C < original_point_C:
    for try_count in range(6): #5回までなら、rateを大きくして試せる。
        try:
            ids_c = Knapsack(size_C, weight_C, capacity_C,  obtained_price_C, rate=rate)
        except TypeError:
            rate += 0.02
        else: # エラーが発生しなかった時の処理
            break
    else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]
else: # リトライが全部失敗した時の処理
        ids_c = [0 for i in range(len(size_C))]

In [90]:
# 視聴率データのインデックス。
# +1するのは、0だと選択しようがしまいが±0で値としては等しくなってしまうから。
ids = np.array(sample['id'].index) + 1

In [91]:
# それぞれが選んだ枠のid『のindex(+1)』が選択されている。
print(sum(ids_a),'個の枠:', ids_a * ids -1)
print(sum(ids_b),'個の枠:', ids_b * ids -1)
print(sum(ids_c),'個の枠:', ids_c * ids -1)

0 個の枠: [-1 -1]
0 個の枠: [-1 -1]
1.0 個の枠: [ 0. -1.]


In [92]:
# それぞれの枠を選択したクライアントの中から１つをランダムに選択する。
ids_dict = dict()
for i in ids-1:
    values = []
    if i in ids_a * ids -1:
        values.append('a')
    if i in ids_b * ids -1:
        values.append('b')
    if i in ids_c * ids -1:
        values.append('c')
    if values:
        value = random.choice(values)
        ids_dict[i] = value

In [93]:
# クライアントごとに、取得することのできた枠のid『のindex』をリスト化する。
a_add_ids = [ids for ids, client in ids_dict.items() if client == 'a']
b_add_ids = [ids for ids, client in ids_dict.items() if client == 'b']
c_add_ids = [ids for ids, client in ids_dict.items() if client == 'c']

# それらの和集合（＝抜き出すもの）
extraction_ids = a_add_ids + b_add_ids + c_add_ids

In [94]:
# 取得することのできた枠のid
print("client_a:", a_add_ids)
print("client_b:", b_add_ids)
print("client_c:", c_add_ids)

client_a: []
client_b: []
client_c: [0]


In [95]:
# 取得した枠のidを足す。
obtained_id_A += a_add_ids
obtained_id_B += b_add_ids
obtained_id_C += c_add_ids

In [96]:
# 取得した枠の値段を足す。
obtained_price_A += sum(sample.query('index in ' + str(a_add_ids))['price_A'])
obtained_price_B += sum(sample.query('index in ' + str(b_add_ids))['price_B'])
obtained_price_C += sum(sample.query('index in ' + str(c_add_ids))['price_C'])

In [97]:
# 取得した枠の視聴率を足す。
obtained_point_A += sum(sample.query('index in ' + str(a_add_ids))['rating_A'])
obtained_point_B += sum(sample.query('index in ' + str(b_add_ids))['rating_B'])
obtained_point_C += sum(sample.query('index in ' + str(c_add_ids))['rating_C'])

In [98]:
# 取得された枠を削除する。
sample = sample.query('index not in ' + str(extraction_ids)).reset_index(drop=True)

In [104]:
print('残りの枠:', len(sample))
print('*'*45)
print('【client_A(価格)】', '交換前:', round(original_price_A,0), '獲得:', round(obtained_price_A,0))
print('【client_B(価格)】', '交換前:', round(original_price_B,0), '獲得:', round(obtained_price_B,0))
print('【client_C(価格)】', '交換前:', round(original_price_C,0), '獲得:', round(obtained_price_C,0))
print('*'*45)
print('【client_A(視聴率)】', '交換前:', round(original_point_A,1), '獲得:', round(obtained_point_A,1), '○' if obtained_point_A > original_point_A else '×')
print('【client_B(視聴率)】', '交換前:', round(original_point_B,1), '獲得:', round(obtained_point_B,1), '○' if obtained_point_B > original_point_B else '×')
print('【client_C(視聴率)】', '交換前:', round(original_point_C,1), '獲得:', round(obtained_point_C,1), '○' if obtained_point_C > original_point_C else '×')
display(sample.head(3))

残りの枠: 1
*********************************************
【client_A(価格)】 交換前: 6751 獲得: 6620
【client_B(価格)】 交換前: 6826 獲得: 6926
【client_C(価格)】 交換前: 5409 獲得: 5460.0
*********************************************
【client_A(視聴率)】 交換前: 65.3 獲得: 67.0 ○
【client_B(視聴率)】 交換前: 68.1 獲得: 72.4 ○
【client_C(視聴率)】 交換前: 53.8 獲得: 54.9 ○


,id,price_A,price_B,price_C,rating_A,rating_B,rating_C,value_A,value_B,value_C
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
# データフレームの長さが1だとうまく動作しないので、追加する。
if len(sample) == 1 and sample['id'][0] != 0:
    add_df = pd.DataFrame(np.zeros(10)).T
    add_df.columns = sample.columns
    sample = pd.concat([sample, add_df]).reset_index(drop=True)

# 上のプログラムの後にデータフレームの長さが1(=add_dfしか残っていない)なら、終了！！
if len(sample) == 1:
    print('finish!!!!')

finish!!!!


***
***

In [101]:
if obtained_point_A >original_point_A and obtained_point_B >original_point_B and obtained_point_C >original_point_C:
    print('Good Flame turn!!!')
    sum_point = obtained_point_A + obtained_point_B + obtained_point_C 
    result[sum_point] = [obtained_id_A, obtained_id_B, obtained_id_C]
else:
    print('Bad Flame turn!! Try again!!')

Good Flame turn!!!


In [102]:
result

{194.3: [[1, 16, 28, 4, 6, 9, 16],
  [13, 5, 8, 14, 21, 12, 14, 0, 4, 7, 8, 10],
  [2, 27, 5, 13, 15, 0, 1, 3, 4, 5, 0]]}

### これを何度も繰り返すことにより、resultに値を格納していく。

In [103]:
# 最も視聴率和を大きくした交換の仕方を求める。
result[max(result.keys())]

[[1, 16, 28, 4, 6, 9, 16],
 [13, 5, 8, 14, 21, 12, 14, 0, 4, 7, 8, 10],
 [2, 27, 5, 13, 15, 0, 1, 3, 4, 5, 0]]

### プロトタイプは完成。あとはエラーの処理をし、アルゴリズム化する。